In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


## 1、深入理解卷积中量化的细节

### 1.1 定义一个输入并且输出指定范围数据

In [2]:
a = torch.randn((64,1,28,28))
a[0][0][0][:5]

tensor([-0.5508,  0.0043, -0.1790, -1.3885,  1.2833])

### 1.2 定义一个卷积

In [3]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 1.3 进行卷积运算并且输出指定范围数据

In [4]:
res = conv(a)
res[0][0][0][:5]

tensor([-0.2279, -0.8645, -0.4443,  0.4254,  0.0018], grad_fn=<SliceBackward>)

> 上面一步的目的是模拟正常运算，此时网络中的权重和偏置已存在

### 1.4 量化
#### 1、 进行对输入的量化

In [5]:
# 1
min_a, max_a = calcu_max_and_min(a, None, None, False)
# 2
scale_a, zero_point_a = calcu_scale_and_zeropoint(min_a, max_a, 8, False)
# 3
q_a = quantize_tensor(a, scale_a, zero_point_a).int()
# 4
dq_a = dequantize_tensor(q_a, scale_a, zero_point_a)

print("原始：", a[0][0][0][:5])
print("量化：", q_a[0][0][0][:5])
print("反量化：", dq_a[0][0][0][:5])

原始： tensor([-0.5508,  0.0043, -0.1790, -1.3885,  1.2833])
量化： tensor([104, 123, 117,  77, 164], dtype=torch.int32)
反量化： tensor([-0.5814,  0.0000, -0.1836, -1.4075,  1.2545])


#### 2、 进行对权重的量化
两种量化方式，逐层量化、逐通道量化

这里展示的是逐通道量化

In [6]:
# 1、获取权重
w = conv.weight.data
# 2、
min_w, max_w = calcu_max_and_min(w, None, None, True)
# 3、
scale_w, zero_point_w = calcu_scale_and_zeropoint(min_w, max_w, 8, True)
# 4、
q_w = quantize_tensor(w, scale_w, zero_point_w,8,False, True)
# 5、
dq_w = dequantize_tensor(q_w, scale_w, zero_point_w, True)

print("原始：", w[0][0][0][:5])
print("量化：", q_w[0][0][0][:5])
print("反量化：", dq_w[0][0][0][:5])

原始： tensor([-0.0702,  0.2863,  0.3016])
量化： tensor([ 99.8421, 247.9393, 254.2934])
反量化： tensor([-0.0702,  0.2863,  0.3016])


#### 3、 进行对输出的量化

In [7]:
# 1
min_res, max_res = calcu_max_and_min(res, None, None, False)
# 2
scale_res, zero_point_res = calcu_scale_and_zeropoint(min_res, max_res, 8, False)
# 3
q_res = quantize_tensor(res, scale_res, zero_point_res).int()
# 4
dq_res = dequantize_tensor(q_res, scale_res, zero_point_res)

print("原始：", res[0][0][0][:5])
print("量化：", q_res[0][0][0][:5])
print("反量化：", dq_res[0][0][0][:5])

原始： tensor([-0.2279, -0.8645, -0.4443,  0.4254,  0.0018], grad_fn=<SliceBackward>)
量化： tensor([129, 105, 121, 153, 138], dtype=torch.int32)
反量化： tensor([-0.2425, -0.8890, -0.4580,  0.4041,  0.0000])


### 1.5 卷积中权值更新为量化权值

In [8]:
conv.weight.data = q_w

### 1.6 进行推理
<img src="https://www.zhihu.com/equation?tex=S_a+%28q_a-Z_a%29%3D%5Csum_%7Bi%7D%5EN+S_w%28q_w-Z_w%29S_x%28q_x-Z_x%29%2BS_b%28q_b-Z_b%29+%5Ctag%7B2%7D+" alt="[公式]" style="zoom:80%;" />
<img src="https://www.zhihu.com/equation?tex=q_a%3D%5Cfrac%7BS_w+S_x%7D%7BS_a%7D%5Csum_%7Bi%7D%5EN+%28q_w-Z_w%29%28q_x-Z_x%29%2B%5Cfrac%7BS_b%7D%7BS_a%7D%28q_b-Z_b%29%2BZ_a+%5Ctag%7B3%7D+" alt="[公式]" style="zoom:80%;" />

#### 1、量化输入

In [9]:
x_1 = q_a - zero_point_a

#### 2、进行卷积
> 注意：
> 1. 这里的输入已经模拟量化了
> 2. 卷积中的权重在上面某步中被模拟量化了（偏置被忽略了）

In [10]:
y_2 = conv(x_1)

#### 3、计算 M

![[公式]](https://www.zhihu.com/equation?tex=M%3D%5Cfrac%7BS_1+S_2%7D%7BS_3%7D)

注意：

> 1. M的计算公式根据逐层量化和逐通道量化方式的不同而不同，这里展示的是逐通道量化的公式

In [11]:
M = [scale_w[i] * scale_a / scale_res for i in range(len(scale_w))]
len(M)

40

#### 4、M 与 量化卷积乘积


In [12]:
for i in range(len(M)):
    y_2[:,i,:,:] = y_2[:,i,:,:] * M[i]

#### 5、加上 输出的zero_point
这里得到的是量化的输出结果

In [13]:
z_3 = y_2 +zero_point_res
    
z_3[0][0][0][:5]

tensor([142.9339, 130.2727, 140.0809, 161.4011, 141.4653],
       grad_fn=<SliceBackward>)

#### 6、反量化

In [14]:
dz_4 = dequantize_tensor(z_3, scale_res, zero_point_res)
dz_4[0][0][0][:5]

tensor([ 0.1329, -0.2082,  0.0561,  0.6304,  0.0934], grad_fn=<SliceBackward>)

#### 7、与原始结果对比

In [15]:
print("原始：", res[0][0][0][:5])
print("量化：", q_res[0][0][0][:5])
print("反量化：", dq_res[0][0][0][:5]) 

原始： tensor([-0.2279, -0.8645, -0.4443,  0.4254,  0.0018], grad_fn=<SliceBackward>)
量化： tensor([129, 105, 121, 153, 138], dtype=torch.int32)
反量化： tensor([-0.2425, -0.8890, -0.4580,  0.4041,  0.0000])


## 2、封装版的量化过程

### 2.1 定义一个输入并且输出指定范围数据

In [16]:
a = torch.randn((64,1,28,28))
a[0][0][0][:5]

tensor([-0.9104, -0.0461,  0.1548,  0.4115,  1.4664])

### 2.2 定义一个卷积

In [17]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 2.3 进行卷积运算并且输出指定范围数据

In [18]:
res = conv(a)
res[0][0][0][:5]

tensor([ 0.4986, -0.0561,  0.2890,  0.5639,  0.2650], grad_fn=<SliceBackward>)

### 2.4 定义一个量化卷积

In [19]:
qconv = QConv2d(conv, True, True, 8, True)

#### 2.5 进行量化卷积运算
> 目的是计算量化中的参数，固定

In [20]:
q_b = qconv(a)
qconv.freeze()

#### 2.6 量化推理

In [21]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][0][:5]

tensor([ 0.4683, -0.0739,  0.2711,  0.5422,  0.2464], grad_fn=<SliceBackward>)

In [22]:
res[0][0][0][:5]

tensor([ 0.4986, -0.0561,  0.2890,  0.5639,  0.2650], grad_fn=<SliceBackward>)